# Histogram matching
Utility scripts for batch histogram matching of photos. Histogram matching can be used to normalize differences in lighting between photos.

In [1]:
import os
import numpy as np
from skimage import io
from skimage.exposure import match_histograms

## Helper functions

In [20]:
def list_paths(file_dir, valid_extensions):
    """
    Return a list of paths to files in a directory with valid extensions
    @param {str} file_dir The directory to find files in
    @param {list} valid_extensions All valid extensions, such as ".tif"
    @return {list} Path to all files with valid extensions in the file directory
    """
    return [os.path.join(file_dir, file) for file in os.listdir(file_dir) if any(file.endswith(ext) for ext in valid_extensions)]

In [13]:
def add_suffix(file_path, suffix):
    """
    Insert a suffix into the file name of a path.
    @param {str} file_path A relative or absolute path to the file, including the file name
    @param {str} suffix A set of characters to append to the file name as a suffix
    @return {str} A relative or absolute path to the file with the suffix appended to the file name
    """
    path, basename = os.path.split(file_path)
    name, ext = os.path.splitext(basename)
    name_suffix = f"{name}{suffix}{ext}"
    
    full_path = os.path.join(path, name_suffix)
    return full_path

## Matching function

In [59]:
def histogram_match_photos(photo_paths, write=True, save_dir=None, save_suffix="_match", reference_index=0, dtype=None):
    """
    Histogram match a set of photos, using one of the photos as a reference. Return the matched photos
    and optionally write to files.
    @param {list} photo_paths A list of relative or absolute paths to image files
    @param {bool, default True} write If true, the matched photos will be written to disk.
    @param {str, default None} save_dir A directory to write matched files to. If none is provided,
    files will be written to the directory they were read from.
    @param {str, default "_match"} saveSuffix Characters to append to the filename when writing
    @param {int, default 0} reference_index The index of the photo path to use as a reference when
    histogram matching.
    @param {numpy.dtype, default None} dtype The data type to cast values of the matched photo to.
    If none is provided, the data type of the input reference image will be used.
    @return
    """
    photos = [io.imread(path) for path in photo_paths]
    reference = photos[reference_index]
    
    if not dtype:
        dtype = reference.dtype
    
    matched_photos = []
    
    for i, photo in enumerate(photos):
        print(f"Matching photo {i + 1} of {len(photos)}...")
        
        # Histogram match and cast pixels to original data type
        matched = match_histograms(photo, reference).astype(dtype)
        matched_photos.append(matched)
        
        if write:
            in_path = photo_paths[i]
            out_path = os.path.join(save_dir, os.path.basename(in_path)) if save_dir else in_path
            out_path = add_suffix(out_path, save_suffix)
            print(f"Writing matched photo to {out_path}...")
            io.imsave(out_path, matched)
    
    return matched_photos

## Example

In [ ]:
img_dir = os.path.join("E:", "ORISE", "Projects", "other", "naficy_photogrammetry", "DWW 1965-69 Six Mile GYE")
save_dir = os.path.join("E:", "ORISE", "Projects", "other", "naficy_photogrammetry", "DWW 1965-69 Six Mile GYE", "hist_matched")
valid_extensions = (".tif")
photo_paths = list_paths(img_dir, valid_extensions)

matched_photos = histogram_match_photos(photo_paths, write=True, save_dir=save_dir, save_suffix="_matched")

Matching photo 1 of 18...
